### import
- 다운자료의 날짜는 전일(기준일)로 한다!
- 기준일 다음날 아침에 계좌별진행상황 다운받기, 틀린 계좌수정하기, 통합하기, 전산 올리기, 그 결과를 엑셀로 저장하기(기준일까지의 진행상황 반영하기)
- 기준일 다음날 아침에 입금 다운받기, 통합하기, 전산 올리기, 그 결과를 엑셀로 저장하기(기준일까지의 진행상황 반영하기)

In [1]:
import os, re, pickle
import pandas as pd
import numpy as np
from os.path import join
from tqdm import tqdm

# dtype 정의 (사이트 파일은 읽지 말자. 할 게 너무 많음)
nauri_dtype = {'채무자키' : str, '계좌키' : str, '보증인키':str, '신용회복키' : str, '심의차수' : str, '조정전이율' : str, '조정후이율' : str, '총상환기간' : str, '유예기간' : str, '원금균등상환기간' : str, '원리균등상환기간' : str, '이자상환기간' : str, '납입회차' : str, '연체기간' : str, '원금균등시작회차' : str, '원금균등종료회차' : str, '원리균등시작회차' : str, '원리균등종료회차' : str, '이자상환시작회차' : str, '이자상환종료회차' : str, '감면율' : str, '주채무자주민번호' : str, '시작회차1' : str, '종료회차1' : str, '채무조정1적용이율1' : str, '채무조정2적용이율1' : str, '시작회차2' : str, '종료회차2' : str, '채무조정1적용이율2' : str, '채무조정2적용이율2' : str, '시작회차3' : str, '종료회차3' : str, '채무조정1적용이율3' : str, '채무조정2적용이율3' : str, '시작회차4' : str, '종료회차4' : str, '채무조정1적용이율4' : str, '채무조정2적용이율4' : str, '시작회차5' : str, '종료회차5' : str, '채무조정1적용이율5' : str, '채무조정2적용이율5' : str, '시작회차6' : str, '종료회차6' : str, '채무조정1적용이율6' : str, '채무조정2적용이율6' : str, '시작회차7' : str, '종료회차7' : str, '채무조정1적용이율7' : str, '채무조정2적용이율7' : str, '시작회차8' : str, '종료회차8' : str, '채무조정1적용이율8' : str, '채무조정2적용이율8' : str, '시작회차9' : str, '종료회차9' : str, '채무조정1적용이율9' : str, '채무조정2적용이율9' : str, '시작회차10' : str, '종료회차10' : str, '채무조정1적용이율10' : str, '채무조정2적용이율10' : str, '재조정횟수' : str, '수정조정횟수' : str, '일시납감면율' : str, '거치기간' : str, '체증전구간의시작회차' : str, '체증전구간의종료회차' : str, '체증전구간의기준기간' : str, '변제금수취계좌' : str, '담보권실행유예기간':str
}
deposit_dtype = {} ####################################

### 파일 읽기

In [2]:
# 파일 읽기
path_base = r"D:\3.자산\신용회복\신용회복 전체데이터\계좌별 진행상황"
########################################################
basedate = "231010"
company = "솔림"
########################################################
sufix = "_업로드결과"

# 입금 업로드 결과
if company == "솔림" : 
    deposit_base = r"D:\3.자산\신용회복\솔림\1.신용회복\0.입금관련,담당자변경\2023년"
    deposit_file = "20"+basedate +"_신용회복입금_통합" + sufix
else : 
    deposit_base = r"D:\3.자산\신용회복\대성\신용회복\입금"
    deposit_file = "대성 신용회복 입금_"+basedate + sufix

# 읽을 파일명
filename_read = basedate + "_계좌별 진행상황 조회_" + company + sufix
fullpath_read = join(path_base, filename_read+".xlsx")

# 계좌별 진행상황 조회 파일 읽기
raw_ori = pd.read_excel(fullpath_read, dtype=nauri_dtype).fillna("")

# 입금 파일 읽기
deposit = pd.read_excel(join(deposit_base, deposit_file), dtype=deposit_dtype).fillna("")

In [3]:
# 재시작 위해 복사본 만들기
raw = raw_ori.copy()

# 불필요한 열 삭제
raw.drop(columns=["채무자명","채무상태", "현재채권구분","등록파일","수정파일", "삭제여부","등록일자","등록유저","수정일자","수정유저","확인결과","저장결과"], axis=1, inplace=True)

# 새로운 열을 추가 : insert 여러번 쓰니 성능 경고나와서
진행구분 = pd.Series([None] * len(raw), name='진행구분')
반영OPB = pd.Series([None] * len(raw), name='반영OPB')
상환구분 = pd.Series([None] * len(raw), name='상환구분')

position = 4
raw = pd.concat([raw.iloc[:, :position], 진행구분, 반영OPB, 상환구분, raw.iloc[:, position:]], axis=1)

# 전산미등록건 처리 (강민경)
미등록주민번호 = ['771027-2221231']
cond = raw['신청인주민번호'].isin(미등록주민번호)
raw.loc[cond, '채무자키'] = '전산미등록건'
raw.loc[cond, '계좌키'] = '전산미등록건'

### 진행구분, 반영OPB, 상환구분 작성

In [4]:
# 진행구분열 작성-------------------------------
x = raw.신청인진행상태
y = raw.계좌진행상태내용
z = raw.계좌번호

conds = [
    (x == '정상이행') & (y == '채무조정포함'),
    (y == "채무조정제외"),
    (x.str.contains(r"완제|상환") | y.str.contains(r"완제|상환") | z.str.contains(r"완제|상환")),
    (z == "채권신고전"),
    (x.str.contains(r"심사반송|중도실효|합의서체결포기")),
]
values = ["확정", "채무조정제외","완제","채권신고전","실효"]
default_value = "미확정"

raw["진행구분"] = np.select(conds, values, default_value)

# 반영OPB - 확정미확정 건-------------------------------
x = raw.진행구분
# 확정시 우선순위
a = raw.상환후잔액
b = raw.원금균등채무액
c= raw.조정후원금
d = raw.조정전원금

conds = [(x == "미확정"), (x =="확정")&(a!=0), b!=0, c!=0, d!=0]

# 조건에 따라 값을 설정
values = [d, a, b, c, d]

# 기본값 설정
default_value = "오류"

# 반영OPB 열 설정
raw["반영OPB"] = np.select(conds, values, default_value)

# 오류건체크 : a,b,c,d 어떤 조건도 만족하지 못한경우
반영OPB오류 = raw[raw.반영OPB == "오류"]
print(f"반영OPB오류건 : {len(반영OPB오류)}")
if len(반영OPB오류) > 0 : display(반영OPB오류)


# 상환구분 작성 - 확정미확정 건-------------------------------
x = raw.감면방식

conds = [x=="원금균등", x==""]

# 조건에 따라 값을 설정
values = ["프리", ""]

# 기본값 설정
default_value = "개인"

# 반영OPB 열 설정
raw["상환구분"] = np.select(conds, values, default_value)

### 시트작성 및 체크

In [5]:
# 최종 시트
최종 = raw.query('진행구분 in ["확정","미확정","실효","완제"]').copy()
확정미확정 = raw.query('진행구분 in ["확정", "미확정"]').copy()

# 전산미등록건 제외
최종.drop(최종[최종.채무자키=='전산미등록건'].index, inplace=True)
확정미확정.drop(확정미확정[확정미확정.채무자키=='전산미등록건'].index, inplace=True)

# 순번열 넣기
raw.insert(0, "순번", range(1,len(raw)+1))
최종.insert(0, "순번", range(1,len(최종)+1))
확정미확정.insert(0, "순번", range(1,len(확정미확정)+1))

In [11]:
# 체크
최종키오류 = 최종.query('채무자키=="" or 계좌키==""')
확정미확정키오류 = 확정미확정.query('채무자키=="" or 계좌키==""')
print(f"전체 : {len(raw)}, 최종 : {len(최종)}, 확정미확정 : {len(확정미확정)}")
print(f"최종에서 키 없는 행 수 : {len(최종키오류)}, 확정미확정에서 키없 는 행 수 : {len(확정미확정키오류)}")
if len(확정미확정키오류)>0 :
    display(확정미확정키오류)
else : print("키오류 없음")

전체 : 18722, 최종 : 18449, 확정미확정 : 14387
최종에서 키 없는 행 수 : 73, 확정미확정에서 키없 는 행 수 : 1


,순번,채무자키,계좌키,보증인키,신용회복키,진행구분,반영OPB,감면구분,신청인명,신청인주민번호,...,체증전구간의종료회차,체증전구간의기준기간,변제금수취계좌,담보권실행유예기간,특별면책,특별상환유예,특례지원,이자율재조정,재난상환유예,거치이자납입유예
1156,276,,,,,미확정,5308104.0,,이상일,620726-1080113,...,0,0,,0,N,,N,N,N,N


### 파일 저장

In [9]:
# 저장
# 기준데이터 저장 경로
filename_write = basedate+"_신용회복_기준데이터_"+company
fullpath_result = join(os.path.dirname(path_base), filename_write+".xlsx")


# 엑셀 파일로 저장
if not os.path.exists(fullpath_result):
    param = {'mode' : 'w'}
else : 
    param = {'mode' : 'a', 'if_sheet_exists' : 'replace'}

with pd.ExcelWriter(fullpath_result, engine='openpyxl', **param) as writer :
    raw.to_excel(writer, sheet_name='원본', index=False)
    최종.to_excel(writer, sheet_name='최종', index=False)
    확정미확정.to_excel(writer, sheet_name='확정,미확정', index=False)